# Obtaining dataset

In [1]:
import os
import torch
from torchvision import datasets, models, transforms

## DATA LOADER
data_dir = './images'
TRAIN = 'train'
VAL = 'val'
TEST = 'test'

# Squeezenet Takes 224x224 images as input, so we resize all of them
data_transforms = {
    TRAIN: transforms.Compose([
        # Data augmentation is a good practice for the train set
        # Here, we randomly crop the image to 224x224 and
        # randomly flip it horizontally.
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    # VAL: transforms.Compose([
    #     transforms.Resize(256),
    #     transforms.CenterCrop(224),
    #     transforms.ToTensor(),
    # ]),
    TEST: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
}

image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x),
        transform=data_transforms[x]
    )
    for x in [TRAIN, TEST]
}

dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=8,
        shuffle=True, num_workers=4
    )
    for x in [TRAIN, TEST]
}

dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, TEST]}

for x in [TRAIN, TEST]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))

print("Classes: ")
class_names = image_datasets[TRAIN].classes
print(image_datasets[TRAIN].classes)


Loaded 16000 images under train
Loaded 2000 images under test
Classes: 
['flower', 'sugarcane']


# Quantization classes and functions

This implementation is based on the code available at https://github.com/aaron-xichen/pytorch-playground. Although the refered repository offers some options for quantizing popular CNN architectures like Squeezenet, VGG, Alexnet and Resnet, a more dedicated code for my application was necessary in my opinion.

In [2]:
import torch
from torch import nn
from torch.autograd import Variable
import math

def compute_integral_part(input, overflow_rate):
    """Calculates the scaling factor (sf) that better represents the input"""
    # transform 'input' into an array of the abs of each elements 
    abs_value = input.abs().view(-1)
    # sort the modulus array in descending order
    sorted_value = abs_value.sort(dim=0, descending=True)[0]
    # find what index corresponds to the max possibe modulus value, considering the overflow_rate.
    # for '0' overflow_rate, the index will be the one of the maximum module of all modules, and 
    # the biggest modulus (index 0) will be chosen
    split_idx = int(overflow_rate * len(sorted_value))
    # value at that index
    v = sorted_value[split_idx]
    #print('v is {}'.format(v))
    if isinstance(v, Variable):
        v = v.data.cpu().numpy()
    # get the minimum ammount of bits required to represent the value chosen and consider it the 
    # scaling factor. The '1e-12' is there to determine the smallest precision (if 'v' is too small)
    sf = math.ceil(math.log2(v+1e-12))
    #print('sf is {}'.format(sf))
    return sf

def linear_quantize(input, sf, bits):
    """Converts a float value from the real numbers domain to a float in the quantized domain"""
    assert bits >= 1, bits
    if bits == 1:
        return torch.sign(input) - 1
    #print('inside sf is {}'.format(sf))
    # calculate the minimum step, considering that the 'sf' bits will quantize in the interval [0,1].
    # this is equivalento to compute 1/(2^(sf)),  or 2^(-sf)
    delta = math.pow(2.0, -sf)
    bound = math.pow(2.0, bits-1)
    # calculates min and maximum. For 8 bits, the quantized number will be between [-128,127]. 
    min_val = - bound
    max_val = bound - 1
    
    # dividing the input by delta and flooring
    ## rounded = torch.floor(input / delta + 0.5) # Equivalent to torch.round(input / delta)
    rounded = torch.round(input / delta)

    clipped_value = torch.clamp(rounded, min_val, max_val) * delta
    return clipped_value


class LinearQuant(nn.Module):
    def __init__(self, name, bits, sf=None, overflow_rate=0.0, counter=10):
        super(LinearQuant, self).__init__()
        self.name = name
        self._counter = counter

        self.bits = bits
        self.sf = sf
        self.overflow_rate = overflow_rate

    @property
    def counter(self):
        return self._counter

    def forward(self, input):
        if self._counter > 0:
            self._counter -= 1
            sf_new = self.bits - 1 - compute_integral_part(input, self.overflow_rate)
            self.sf = min(self.sf, sf_new) if self.sf is not None else sf_new
            return input
        else:
            output = linear_quantize(input, self.sf, self.bits)
            return output

    def __repr__(self):
        return '{}(sf={}, bits={}, overflow_rate={:.3f}, counter={})'.format(
self.__class__.__name__, self.sf, self.bits, self.overflow_rate, self.counter)


def duplicate_model_with_quant(model, bits, overflow_rate=0.0, counter=10):
    """assume that original model has at least a nn.Sequential"""
    if isinstance(model, nn.Sequential):
        l = OrderedDict()
        for k, v in model._modules.items():
            if isinstance(v, (nn.Conv2d, nn.Linear, nn.BatchNorm1d, nn.BatchNorm2d, nn.AvgPool2d)):
                l[k] = v
#                if type == 'linear':
#                    quant_layer = LinearQuant('{}_quant'.format(k), bits=bits, overflow_rate=overflow_rate, counter=counter)
#                elif type == 'log':
#                    # quant_layer = LogQuant('{}_quant'.format(k), bits=bits, overflow_rate=overflow_rate, counter=counter)
#                    quant_layer = NormalQuant('{}_quant'.format(k), bits=bits, quant_func=log_minmax_quantize)
#                elif type == 'minmax':
#                    quant_layer = NormalQuant('{}_quant'.format(k), bits=bits, quant_func=min_max_quantize)
#                else:
#                    quant_layer = NormalQuant('{}_quant'.format(k), bits=bits, quant_func=tanh_quantize)
                quant_layer = LinearQuant('{}_quant'.format(k), bits=bits, overflow_rate=overflow_rate, counter=counter)
                l['{}_{}_quant'.format(k, type)] = quant_layer
            else:
                l[k] = duplicate_model_with_quant(v, bits, overflow_rate, counter)
        m = nn.Sequential(l)
        return m
    else:
        for k, v in model._modules.items():
            model._modules[k] = duplicate_model_with_quant(v, bits, overflow_rate, counter)
    return model

def eval_model(squeezenet, criterion, verbose=False):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0

    test_batches = len(dataloaders[TEST])
    if verbose:
        print("Evaluating model")
        print('-' * 10)

    squeezenet.train(False)
    squeezenet.eval()

    with torch.no_grad():
        for i, data in enumerate(dataloaders[TEST]):
            if verbose:
                if i % 10 == 0:
                    print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)


            inputs, labels = data
            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()
            else:
                inputs, labels = inputs, labels

            outputs = squeezenet(inputs)

            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)

            loss_test += loss.data
            acc_test += torch.sum(preds == labels.data).item()

            # del inputs, labels, outputs, preds
            # torch.cuda.empty_cache()

    avg_loss = loss_test / dataset_sizes[TEST]
    avg_acc = acc_test / dataset_sizes[TEST]

    elapsed_time = time.time() - since
    if verbose:
        print()
        print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
        print("Avg loss (test): {:.4f}".format(avg_loss))
        print("Avg acc (test): {:.4f}".format(avg_acc))
        print('-' * 10)
    return avg_acc, avg_loss



In [3]:
x = torch.ones(3, 3)
x[0,2] = 1
x[0,2] = 3.6
abs_values = x.abs().view(-1)
sorted_values = abs_values.sort(dim=0, descending=True)[0]
print(sorted_values)

overflow_rate = 0.0
split_idx = int(overflow_rate * len(sorted_values))
print(split_idx)

v = sorted_values[split_idx]
v = 5
print("v is {}".format(v))
a = math.log2(v+1e-12)
print(a)

sf = compute_integral_part(x, 0.0)
print(sf)

tensor([3.6000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
0
v is 5
2.321928094887651
2


In [4]:
bits = 8

x = torch.ones(1, 1)
x[0,0] = 25

sf = compute_integral_part(x, 0.0)
print('sf is {}'.format(sf))

sf = bits - 1. - sf
print('post-calculated sf is {}'.format(sf))

delta = math.pow(2.0, -sf)
print('delta is {}'.format(delta))

a = torch.Tensor(1)
a[0] = 32
quantized = torch.round(a / delta)
print('a ({}) quantized is {}'.format(a.numpy(), quantized.numpy()))
rounded = quantized*delta
print('a ({}) rounded is {}'.format(a.numpy(), rounded.numpy()))

sf is 5
post-calculated sf is 2.0
delta is 0.25
a ([32.]) quantized is [128.]
a ([32.]) rounded is [32.]


In [5]:
import argparse
import torch
import torch.backends.cudnn as cudnn
cudnn.benchmark =True
from collections import OrderedDict

import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn

#parser = argparse.ArgumentParser(description='PyTorch SVHN Example')
# parser.add_argument('--type', default='cifar10', help='|'.join(selector.known_models))
# parser.add_argument('--quant_method', default='linear', help='linear|minmax|log|tanh')
#parser.add_argument('--batch_size', type=int, default=100, help='input batch size for training (default: 64)')
#parser.add_argument('--gpu', default=None, help='index of gpus to use')
#parser.add_argument('--ngpu', type=int, default=8, help='number of gpus to use')
#parser.add_argument('--seed', type=int, default=117, help='random seed (default: 1)')
# parser.add_argument('--model_root', default='~/.torch/models/', help='folder to save the model')
# parser.add_argument('--data_root', default='/tmp/public_dataset/pytorch/', help='folder to save the model')
# parser.add_argument('--logdir', default='log/default', help='folder to save to the log')

#parser.add_argument('--input_size', type=int, default=224, help='input size of image')
# parser.add_argument('--n_sample', type=int, default=20, help='number of samples to infer the scaling factor')
#parser.add_argument('--param_bits', type=int, default=8, help='bit-width for parameters')
#parser.add_argument('--bn_bits', type=int, default=32, help='bit-width for running mean and std')
#parser.add_argument('--fwd_bits', type=int, default=8, help='bit-width for layer output')
# parser.add_argument('--overflow_rate', type=float, default=0.0, help='overflow rate')
# args = parser.parse_args()

args = {}

args['batch_size'] = 100
args['gpu'] = None
args['ngpu'] = 8
args['seed'] = 117
args['input_size'] = 224
#args['param_bits'] = 2
#args['bn_bits'] = 2
#args['fwd_bits'] = 2
args['overflow_rate'] = 0.0

#args.gpu = misc.auto_select_gpu(utility_bound=0, num_gpu=args.ngpu, selected_gpus=args.gpu)
#args.ngpu = len(args.gpu)
#misc.ensure_dir(args.logdir)
#args.model_root = misc.expand_user(args.model_root)
#args.data_root = misc.expand_user(args.data_root)
#args.input_size = 299 if 'inception' in args.type else args.input_size
#assert args.quant_method in ['linear', 'minmax', 'log', 'tanh']
#print("=================FLAGS==================")
#for k, v in args.__dict__.items():
#    print('{}: {}'.format(k, v))
#print("========================================")

#assert torch.cuda.is_available(), 'no cuda'
torch.manual_seed(args['seed'])
torch.cuda.manual_seed(args['seed'])

use_gpu = torch.cuda.is_available()
# use_gpu = False
if use_gpu:
    print("Using CUDA")

In [10]:

def create_model():
    # load model and dataset fetcher
    # model_raw, ds_fetcher, is_imagenet = selector.select(args.type, model_root=args.model_root)
    #squeezenet1_1 = models.squeezenet1_1()

    # importing fixed version of squeezenet class and functions
    import squeezenet_fix

    squeezenet1_1 = squeezenet_fix.squeezenet1_1()

    # Freeze training for all layers
    for param in squeezenet1_1.features.parameters():
        param.require_grad = False

    # Newly created modules have require_grad=True by default
    num_features = squeezenet1_1.classifier[1].in_channels
    features = list(squeezenet1_1.classifier.children())[:-3] # Remove last 3 layers
    features.extend([nn.Conv2d(num_features, 2, kernel_size=1)]) # Add
    features.extend([nn.ReLU(inplace=True)]) # Add
    features.extend([nn.AdaptiveAvgPool2d(output_size=(1,1))]) # Add our layer with 2 outputs
    squeezenet1_1.classifier = nn.Sequential(*features) # Replace the model classifier

    if use_gpu:
        squeezenet1_1.load_state_dict(torch.load('./weights/squeezenet_v1-flower-or-crops.pt'))
    else:
        squeezenet1_1.load_state_dict(torch.load('./weights/squeezenet_v1-flower-or-crops.pt', map_location='cpu'))
    #print(squeezenet1_1)
    return squeezenet1_1


In [11]:
def quantize_state_dict(state_dict, bn_bits, param_bits):
    if param_bits < 32:
        state_dict_quant = OrderedDict()
        sf_dict = OrderedDict()
        for k, v in state_dict.items():
            if 'running' in k:
                if bn_bits >=32:
                    print("Ignoring {}".format(k))
                    state_dict_quant[k] = v
                    continue
                else:
                    bits = bn_bits
            else:
                bits = param_bits

    #        if args.quant_method == 'linear':
    #            sf = bits - 1. - quant.compute_integral_part(v, overflow_rate=args.overflow_rate)
    #            v_quant  = quant.linear_quantize(v, sf, bits=bits)
    #        elif args.quant_method == 'log':
    #            v_quant = quant.log_minmax_quantize(v, bits=bits)
    #        elif args.quant_method == 'minmax':
    #            v_quant = quant.min_max_quantize(v, bits=bits)
    #        else:
    #            v_quant = quant.tanh_quantize(v, bits=bits)
    
            # The sf will be used to do the quantization. Subtract 1 for dividind the range by 2
            # (2^(-sf) will be calculated after), so half of the quatized range represents positive
            # numbers and the other half negative numbers. Subtract the ammount of bits required to
            # represent the max abs value of the input to adjust the scale. At the end of the day, 
            # the operation done through these steps is equivalent to consider that you have "bits - 1"
            # bits to quantize the maximum modulus of the input array. I don't know why to make such simple
            # operation not explicit...
            sf = bits - 1. - compute_integral_part(v, overflow_rate=args['overflow_rate'])
            #sf = compute_integral_part(v, overflow_rate=args['overflow_rate'])
            v_quant  = linear_quantize(v, sf, bits=bits)     
            state_dict_quant[k] = v_quant
        return state_dict_quant

In [12]:
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import time


def quantize_model_forward_activation(model, fwd_bits):
    # Quantize the forward activaton of parameters on the model
    if fwd_bits < 32:
        model = duplicate_model_with_quant(model, bits=fwd_bits)
        #print(squeezenet1_1)
        #val_ds_tmp = ds_fetcher(10, data_root=args.data_root, train=False, input_size=args.input_size)
        #misc.eval_model(model_raw, val_ds_tmp, ngpu=1, n_sample=args.n_sample, is_imagenet=is_imagenet)
        if use_gpu:
            model.cuda() #.cuda() will move everything to the GPU side
    return model

# Analyse quantization vs accuracy

Analyse the accuracy on the test set over different levels of quantization. With this information, it is possible to choose the most suitable quantization level for the application.

In [ ]:
bits_list = list(range(1, 32))
#bits_list = list(range(8, 9))

test_accuracies_list = []
test_losses_list = []

for idx, bits_item in enumerate(bits_list):
    print("\rRunning Tests {}/{}".format(idx+1, len(bits_list)), end='', flush=True)
    # Quantize weights
    squeezenet1_1 = create_model()
    state_dict = squeezenet1_1.state_dict()
    state_dict_quant = quantize_state_dict(state_dict, bits_item, bits_item)
    #print(state_dict_quant)
    squeezenet1_1.load_state_dict(state_dict_quant)

    # Quantize forward activation
    squeezenet1_1_quant = quantize_model_forward_activation(squeezenet1_1, bits_item)
    #print()
    #print(squeezenet1_1_quant)

    # evaluate
    criterion = nn.CrossEntropyLoss()
    #optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    #exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
    avg_acc, avg_loss = eval_model(squeezenet1_1_quant, criterion)
    test_accuracies_list.append(avg_acc)
    test_losses_list.append(avg_loss)
    
    #print()
    #print(idx)
    #print(bits_item)
    #print(avg_acc)
    #print(test_accuracies_list)


Running Tests 6/31

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Data for plotting
t = np.array(bits_list)
s = np.array(test_accuracies_list)

fig, ax = plt.subplots()
ax.plot(t, s)
plt.plot(t, s, 'C0o', alpha=0.5)

ax.set(xlabel='quantization (bits)', ylabel='accuracy',
       title='Accuracy curve for different quantization levels')
ax.grid()

fig.savefig("acc_over_bits.png")
plt.show()